In [71]:
# first, import the circuit benchmarks

# second, run transpilation on benchmarks for many iterations and save results
# want to show how much variance there is in the results
# NOTE important, use the same placement and routing before passed to transpiler so has a fair comparison

# third, use longest path to convert to fidelities
# we want to calculate expected fidelity improvement factor
# is fidelity improvement factor the same as the ratio between lengths (?)
# 

In [72]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR) # turn off for now

#  CONSTANTS
# repeat 10 times, only keep best
data_repitition = range(1)

# 1Q:2Q gate duraton ratio
duration_1q =0.2

In [73]:
from slam.utils.transpiler_pass.speed_limit_pass import \
    pass_manager_basic, \
    pass_manager_optimized_sqiswap
    # pass_manager_slam

""" 
Basic, uses alibaba rules
Optimized, use parallel gate optimized CPhase-family and SWAP ruless
"""
speed_method = 'linear' # just using iswap-family gates the SLF makes no difference for now because they all scale linearly to each other
pm_basic = pass_manager_basic(gate='sqiswap', duration_1q=duration_1q)
pm_optimized = pass_manager_optimized_sqiswap(duration_1q=duration_1q, speed_method=speed_method)

In [74]:
import numpy as np
def fidelity(qc_duration):
    """simple model"""
    iswap_time = 1e-7 # normalize gate duration, iswap:=1 in previous units
    t1 = 1e-4 # coherence time for calc fidelity
    return np.exp(-qc_duration*iswap_time/t1)

def infidelity_factor(qc_time1, qc_time2):
    """Note, for these chosen values, given a fidelity threshold of 0.5 fidelity, duration <= 69.3"""
    if1 = 1 - fidelity(qc_time1)
    if2 = 1 - fidelity(qc_time2)
    # percent reduction
    return 100*(if1-if2)/if1

In [75]:
# first, import the circuit benchmarks
from slam.utils.circuit_suite import benchmark_lambdas

# define the topology
from qiskit import transpile
from qiskit.transpiler import CouplingMap
coupling_map = CouplingMap.from_grid(4,4)
induce_swaps = lambda qc: transpile(qc, coupling_map=coupling_map, optimization_level=3)

# repeat, only keep best
for benchmark_circuit in benchmark_lambdas:

    # best, defined by shortest duration
    basic_best = (None, None, None, None)
    optimized_best = (None, None, None, None)

    for _ in data_repitition:
        qc = benchmark_circuit(16) # instantiate of size 16

        # before inducing, turn off logging for readability
        logger.setLevel(logging.ERROR)
        qc = induce_swaps(qc) # transpile to induce swaps

        # second, run the benchmark
        qc_basic = pm_basic.run(qc)
        duration = pm_basic.property_set['duration']
        gate_counts = pm_basic.property_set['gate_counts']
        crit_counts = pm_basic.property_set['longest_path_counts']

        # check if best
        if basic_best[0] is None or duration < basic_best[0]:
            basic_best = (duration, gate_counts, crit_counts, qc_basic)
        
        # repeat on optimized
        qc_optimized = pm_optimized.run(qc)
        duration = pm_optimized.property_set['duration']
        gate_counts = pm_optimized.property_set['gate_counts']
        crit_counts = pm_optimized.property_set['longest_path_counts']

        # check if best
        if optimized_best[0] is None or duration < optimized_best[0]:
            optimized_best = (duration, gate_counts, crit_counts, qc_optimized)
            
    # print results
    print('basic', basic_best[0])
    print('optimized', optimized_best[0])
    print("duration % reduction", 100*(basic_best[0] - optimized_best[0])/basic_best[0])
    print("basic fidelity -> optimized fidelity", fidelity(basic_best[0]), fidelity(optimized_best[0]))
    print("infidelity % reduction", infidelity_factor(basic_best[0], optimized_best[0]))
    print("\n")
    assert basic_best[0] >= optimized_best[0]

basic 146.50000000000006
optimized 116.60000000000021
duration % reduction 20.409556313993065
basic fidelity -> optimized fidelity 0.8637257323347622 0.8899410984169819
infidelity % reduction 19.237209292233036


basic 29.59999999999998
optimized 25.39999999999999
duration % reduction 14.189189189189161
basic fidelity -> optimized fidelity 0.9708337894079228 0.974919866077912
infidelity % reduction 14.009624791981816


basic 378.8999999999963
optimized 315.4999999999987
duration % reduction 16.732647136447138
basic fidelity -> optimized fidelity 0.6846140706503896 0.7294240710403306
infidelity % reduction 14.207989710367952


basic 174.4999999999996
optimized 152.94392094
duration % reduction 12.353053902578596
basic fidelity -> optimized fidelity 0.8398768542292108 0.8581778462689256
infidelity % reduction 11.429323319635527


basic 415.599999999995
optimized 300.6744216799981
duration % reduction 27.652930298363405
basic fidelity -> optimized fidelity 0.659944195374045 0.740318765253